# Editting_distance
+ 分析子问题的重复性
+ 子问题进行存储
+ Solution 要进行解析

In [5]:
from collections import defaultdict, Counter
from functools import wraps

In [2]:
price=[1, 5, 8, 9, 10, 17, 17, 20, 24, 30]

In [3]:
revenue = defaultdict(int)
for i, value in enumerate(price):
    revenue[i+1] = value

In [4]:
call_time_dic=defaultdict(int)
calculated_node_dic=defaultdict(int)

In [5]:
def call_time(f):
    call_time_dic=defaultdict(int)
    @wraps(f)
    def inner(arg):
        '''call_time doc'''
        call_time_dic[f.__name__ + str(arg)] +=1
        result=f(arg)
        return result
    return inner

In [6]:
def memo(f):
    calculated_node_dic=defaultdict(int)
    @wraps(f)
    def inner(num):
        '''memo doc'''
        if f.__name__ + str(num) in calculated_node_dic:
            return calculated_node_dic[f.__name__+str(num)]
        else:
            result = f(num)
            calculated_node_dic[f.__name__+str(num)] = result
            return result
    return inner

In [7]:
def parse_solution(n):
    left, right = solution[n]
    if left == 0 : return [right]
    else:
        return parse_solution(left) + parse_solution(right)

In [8]:
#get best revenue
solution = {}
@call_time
@memo
def r(lengh):
    '''this is the first decorated function'''
    max_revene, split_len = max([(revenue[lengh],0)] + [(r(i) + r(lengh-i),i) for i in range(1,lengh)],key=lambda x:x[0] )
    solution[lengh] = (split_len,lengh-split_len)
    return max_revene



In [9]:
r(12)

35

In [10]:
parse_solution(12)

[2, 10]

# Editting Distance

In [6]:
import networkx as nx

In [120]:
# calculate edit distance
solution = {}
# @lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [121]:
edit_distance('helloray','hellochen')

4

In [122]:
solution

{('h', 'h'): '',
 ('h', 'he'): 'ADD e',
 ('h', 'hel'): 'ADD l',
 ('h', 'hell'): 'ADD l',
 ('h', 'hello'): 'ADD o',
 ('h', 'helloc'): 'ADD c',
 ('h', 'helloch'): 'ADD h',
 ('h', 'helloche'): 'ADD e',
 ('h', 'hellochen'): 'ADD n',
 ('he', 'h'): 'DEL e',
 ('he', 'he'): '',
 ('he', 'hel'): 'ADD l',
 ('he', 'hell'): 'ADD l',
 ('he', 'hello'): 'ADD o',
 ('he', 'helloc'): 'ADD c',
 ('he', 'helloch'): 'ADD h',
 ('he', 'helloche'): 'ADD e',
 ('he', 'hellochen'): 'ADD n',
 ('hel', 'h'): 'DEL l',
 ('hel', 'he'): 'DEL l',
 ('hel', 'hel'): '',
 ('hel', 'hell'): 'ADD l',
 ('hel', 'hello'): 'ADD o',
 ('hel', 'helloc'): 'ADD c',
 ('hel', 'helloch'): 'ADD h',
 ('hel', 'helloche'): 'ADD e',
 ('hel', 'hellochen'): 'ADD n',
 ('hell', 'h'): 'DEL l',
 ('hell', 'he'): 'DEL l',
 ('hell', 'hel'): 'DEL l',
 ('hell', 'hell'): '',
 ('hell', 'hello'): 'ADD o',
 ('hell', 'helloc'): 'ADD c',
 ('hell', 'helloch'): 'ADD h',
 ('hell', 'helloche'): 'ADD e',
 ('hell', 'hellochen'): 'ADD n',
 ('hello', 'h'): 'DEL o',
 ('h

In [123]:
#find edit sequence of minmun edit distance
min_string_tuple_list=[]
def get_min_edited_solution(str_tuple, result_solution):
    if str_tuple in list(result_solution.keys()): 
        min_string_tuple_list.append([str_tuple, result_solution[str_tuple]])
    else:
        return min_string_tuple_list
    if result_solution[str_tuple] == '':
        str_tuple = (str_tuple[0][:-1], str_tuple[1][:-1])
    elif 'ADD' in result_solution[str_tuple]:
        str_tuple = (str_tuple[0], str_tuple[1][:-1])
    elif 'DEL' in result_solution[str_tuple]:
        str_tuple = (str_tuple[0][:-1], str_tuple[1])
    elif 'SUB' in result_solution[str_tuple]:
        str_tuple = (str_tuple[0][:-1], str_tuple[1][:-1])
    return get_min_edited_solution(str_tuple, result_solution)

In [124]:
get_min_edited_solution( list(solution.keys())[-1],solution)

[[('helloray', 'hellochen'), 'ADD n'],
 [('helloray', 'helloche'), 'SUB y => e'],
 [('hellora', 'helloch'), 'SUB a => h'],
 [('hellor', 'helloc'), 'SUB r => c'],
 [('hello', 'hello'), ''],
 [('hell', 'hell'), ''],
 [('hel', 'hel'), ''],
 [('he', 'he'), ''],
 [('h', 'h'), '']]